# Introduction

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.cross_decomposition import PLSRegression, PLSSVD
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, accuracy_score, confusion_matrix

In [3]:
data = pd.read_csv('data/red_normal.csv') #<---
#data = pd.read_csv('data/red_data.csv')
#data = pd.read_csv('data/white_normal.csv')
#data = pd.read_csv('data/white_data.csv')
#data = pd.read_csv('data/wine_normal.csv')
#data = pd.read_csv('data/wine_data.csv')
data.head()

,fixed_acidity,volatile_acidity,citric,sugar,chlorides,free_SD,total_SD,density,pH,sulphates,alcohol,quality
0,-0.528194,0.981974,-1.391037,-0.590653,-0.231744,-0.247049,-0.159545,0.558100,1.288240,-0.579025,-0.959946,5
1,-0.298454,1.693658,-1.391037,0.296646,0.535000,0.992454,0.854271,0.028252,-0.719708,0.128910,-0.584594,5
2,-0.298454,1.235222,-1.185699,-0.052744,0.342271,0.214135,0.523767,0.134222,-0.331073,-0.048074,-0.584594,5
3,1.654339,-1.592344,1.483689,-0.590653,-0.271015,0.402953,0.684389,0.664069,-0.978798,-0.461036,-0.584594,6
4,-0.528194,0.981974,-1.391037,-0.590653,-0.231744,-0.247049,-0.159545,0.558100,1.288240,-0.579025,-0.959946,5


In [4]:
data.shape

(1599, 12)

In [5]:
X, y = data.ix[:,:-1], data['quality']

In [6]:
pls_flag = True
interaction_flag = True

Our approach is to think about the train as points sitting in $\mathbb R^{12}$ and we conjecture that they have some structure in the form of lower dimensional shape. To have a more concrete visualization, think about $\mathbb R^3$ an our train might have the shape of a sphere, a torus, a bent sheet of paper etc.

Obviously, the train is going to be pretty irregular and we don't expect it to form a really nice submanifold. Nonetheless, we draw from the notion of a triangulation from geometry. Roughly, it says that we can approximate any shape by only using linear pieces without changing the geometric properties too much.

To translate this into our setting, we are going to divide the ranges of the features into a grid and for each small piece we are going to find a linear model to predict the quality. Given a new example, we only need to find where it lies on the grid and make a prediction using the closest model.

In [7]:
# Optionally generate interaction features
if interaction_flag:
    from itertools import combinations_with_replacement
    from functools import reduce
    old_features = [name for name in X.columns if not 'type' in name]
    for combination in combinations_with_replacement(old_features, 4): # for red degree = 3 works best
        temp = reduce(lambda x, y: x*y, map(lambda x: X[x], combination))
        X.insert(X.shape[1]-1, '*'.join(combination), temp) 

In [8]:
X.head()

,fixed_acidity,volatile_acidity,citric,sugar,chlorides,free_SD,total_SD,density,pH,sulphates,...,pH*sulphates*sulphates*sulphates,pH*sulphates*sulphates*alcohol,pH*sulphates*alcohol*alcohol,pH*alcohol*alcohol*alcohol,sulphates*sulphates*sulphates*sulphates,sulphates*sulphates*sulphates*alcohol,sulphates*sulphates*alcohol*alcohol,sulphates*alcohol*alcohol*alcohol,alcohol*alcohol*alcohol*alcohol,alcohol
0,-0.528194,0.981974,-1.391037,-0.590653,-0.231744,-0.247049,-0.159545,0.558100,1.288240,-0.579025,...,-0.250086,-0.414609,-0.687366,-1.139559,0.112406,0.186354,0.308950,0.512198,0.849155,-0.959946
1,-0.298454,1.693658,-1.391037,0.296646,0.535000,0.992454,0.854271,0.028252,-0.719708,0.128910,...,-0.001542,0.006992,-0.031707,0.143787,0.000276,-0.001252,0.005679,-0.025754,0.116793,-0.584594
2,-0.298454,1.235222,-1.185699,-0.052744,0.342271,0.214135,0.523767,0.134222,-0.331073,-0.048074,...,0.000037,0.000447,0.005439,0.066144,0.000005,0.000065,0.000790,0.009604,0.116793,-0.584594
3,1.654339,-1.592344,1.483689,-0.590653,-0.271015,0.402953,0.684389,0.664069,-0.978798,-0.461036,...,0.095918,0.121624,0.154219,0.195550,0.045179,0.057287,0.072641,0.092108,0.116793,-0.584594
4,-0.528194,0.981974,-1.391037,-0.590653,-0.231744,-0.247049,-0.159545,0.558100,1.288240,-0.579025,...,-0.250086,-0.414609,-0.687366,-1.139559,0.112406,0.186354,0.308950,0.512198,0.849155,-0.959946


In [9]:
# Normalize data
if "type_red" in X.columns:
    temp_type = X['type_red']
    X.drop('type_red', axis=1, inplace=True)

X = (X - X.mean())/X.std()

if "type_red" in X.columns:
    X['type_red'] = temp_type

In [10]:
X.head()

,fixed_acidity,volatile_acidity,citric,sugar,chlorides,free_SD,total_SD,density,pH,sulphates,...,pH*sulphates*sulphates*sulphates,pH*sulphates*sulphates*alcohol,pH*sulphates*alcohol*alcohol,pH*alcohol*alcohol*alcohol,sulphates*sulphates*sulphates*sulphates,sulphates*sulphates*sulphates*alcohol,sulphates*sulphates*alcohol*alcohol,sulphates*alcohol*alcohol*alcohol,alcohol*alcohol*alcohol*alcohol,alcohol
0,-0.528194,0.981974,-1.391037,-0.590653,-0.231744,-0.247049,-0.159545,0.558100,1.288240,-0.579025,...,0.068611,-0.153110,-0.245561,-0.232371,-0.077908,0.089980,-0.191705,0.049047,-0.172123,-0.959946
1,-0.298454,1.693658,-1.391037,0.296646,0.535000,0.992454,0.854271,0.028252,-0.719708,0.128910,...,0.072207,-0.095809,0.032496,-0.074423,-0.078509,0.079763,-0.289022,-0.087131,-0.225935,-0.584594
2,-0.298454,1.235222,-1.185699,-0.052744,0.342271,0.214135,0.523767,0.134222,-0.331073,-0.048074,...,0.072230,-0.096698,0.048250,-0.083979,-0.078510,0.079835,-0.290591,-0.078180,-0.225935,-0.584594
3,1.654339,-1.592344,1.483689,-0.590653,-0.271015,0.402953,0.684389,0.664069,-0.978798,-0.461036,...,0.073617,-0.080229,0.111345,-0.068052,-0.078268,0.082951,-0.267535,-0.057295,-0.225935,-0.584594
4,-0.528194,0.981974,-1.391037,-0.590653,-0.231744,-0.247049,-0.159545,0.558100,1.288240,-0.579025,...,0.068611,-0.153110,-0.245561,-0.232371,-0.077908,0.089980,-0.191705,0.049047,-0.172123,-0.959946


In [11]:
if pls_flag:
    pls = PLSRegression(n_components=30, scale=False)
    pls.fit(X, y)
    X = pd.DataFrame(pls.transform(X))

In [12]:
data = pd.concat([X, y], axis=1)

In [13]:
train = data.sample(frac=0.8).sort_index()
train_index = train.index
test = data.ix[~data.index.isin(train_index)].copy(True)

In [14]:
correlations = pd.DataFrame(train.corr()['quality']).apply(np.abs)
temp = correlations.sort_values(by='quality', ascending=False)[1:4]
temp

,quality
0,0.316375
2,0.281363
1,0.262928


In [15]:
train.rename(columns={temp.index[0]:'split_a',temp.index[1]:'split_b', temp.index[2]: 'split_c' }, inplace=True)
test.rename(columns={temp.index[0]:'split_a',temp.index[1]:'split_b', temp.index[2]: 'split_c' }, inplace=True)

In [16]:
X_train, y_train = train.ix[:,:-1], train.quality
X_test, y_test = test.ix[:,:-1], test.quality

In [17]:
clf = LogisticRegression(penalty='l2', C=10, n_jobs=-1)
clf.fit(X_train, y_train)
mean_absolute_error(y_test, clf.predict(X_test))

0.32500000000000001

In [18]:
# Dictionary representing the gird. Each row represents a feature and the columns the partitions.
# The shape of this matrix is a hyperparameter of our model.
# The distance notion that we use is also a hyperparameter.
grid = {}
for name in ['split_'+ char for char in ['a', 'b', 'c']]: #train.columns:
    grid[name] = []
# For each feature we just give one separation point.
for element in grid:
    grid[element] = np.median(train[element])
grid

{'split_a': -1.2475681886457817,
 'split_b': 0.22890151186109223,
 'split_c': -1.6616278857900315}

This generates a split of the train into $2^{12}$ pieces. That is 4096 different pieces so we don't have enough train to support this partition. We need to select a subset of the feautes.

In [19]:
"""tick_marks = [i for i in range(len(train.columns))]
plt.imshow(train.corr(), interpolation='nearest')
plt.colorbar()
plt.xticks(tick_marks, train.columns, rotation='vertical')
plt.yticks(tick_marks, train.columns)
plt.show()"""

"tick_marks = [i for i in range(len(train.columns))]\nplt.imshow(train.corr(), interpolation='nearest')\nplt.colorbar()\nplt.xticks(tick_marks, train.columns, rotation='vertical')\nplt.yticks(tick_marks, train.columns)\nplt.show()"

It seems reasonable to split along alcohol, density and volatile_acidity. This way we will have 8 different partitions. This may change depending on the preprocessing of the data.

In [20]:
dfs = []
dfs.append(train[(train.split_a > grid["split_a"]) 
                 & (train.split_b > grid["split_b"]) 
                 & (train.split_c > grid["split_c"])])
dfs.append(train[(train.split_a > grid["split_a"]) 
                 & (train.split_b > grid["split_b"] ) 
                 & (train.split_c < grid["split_c"])])
dfs.append(train[(train.split_a > grid["split_a"]) 
                 & (train.split_b < grid["split_b"]) 
                 & (train.split_c > grid["split_c"])])
dfs.append(train[(train.split_a > grid["split_a"]) 
                 & (train.split_b < grid["split_b"]) 
                 & (train.split_c < grid["split_c"])])
dfs.append(train[(train.split_a < grid["split_a"])
                 & (train.split_b > grid["split_b"]) 
                 & (train.split_c > grid["split_c"])])
dfs.append(train[(train.split_a < grid["split_a"]) 
                 & (train.split_b > grid["split_b"])
                 & (train.split_c < grid["split_c"])])
dfs.append(train[(train.split_a < grid["split_a"]) 
                 & (train.split_b < grid["split_b"]) 
                 & (train.split_c > grid["split_c"])])
dfs.append(train[(train.split_a < grid["split_a"]) 
                 & (train.split_b < grid["split_b"]) 
                 & (train.split_c < grid["split_c"])])

In [21]:
for df in dfs:
    print(df.shape)

(381, 31)
(28, 31)
(170, 31)
(59, 31)
(45, 31)
(183, 31)
(42, 31)
(368, 31)


In [22]:
def trainLG(df):
    X, y = df.ix[:,:-1], df['quality']
    clf = LogisticRegression(penalty='l2', C=10, n_jobs=-1)
    clf.fit(X, y)
    return clf

In [23]:
clfs = []
for df in dfs:
    clfs.append(trainLG(df))

In [24]:
X_test.head()

,split_a,split_c,split_b,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
3,-1.555519,-1.950545,0.914636,2.567142,2.638137,0.799670,-0.859453,-0.566490,-0.615480,0.370947,...,-0.910964,-1.173742,-1.468858,-0.562162,0.333830,-0.369856,0.408950,1.255754,-0.267939,1.780407
6,-1.240276,-1.852070,0.538802,-0.764130,-1.350139,-1.965743,-1.356869,-0.961592,-1.743965,1.267846,...,0.275995,-0.409521,0.138785,-0.185034,-0.029905,0.529878,-0.185296,-0.661687,0.204157,-0.147077
8,-1.934417,-2.710839,-0.515871,-0.471903,-0.875078,-0.917104,-0.529528,-0.085305,-1.090752,0.800381,...,0.329171,0.144408,0.717904,-0.622600,-0.329310,0.585613,0.108161,0.010893,0.014209,-0.307929
11,-1.682804,-2.199949,-0.107956,-0.542672,-0.076116,0.832077,0.785671,0.376709,-0.566937,1.271838,...,0.543039,0.417775,0.546078,0.332745,0.105123,0.141675,0.882439,0.081475,-0.173375,0.213253
12,1.600006,0.367939,-0.970042,-1.433279,-0.481430,-2.585388,-2.699047,-2.634464,0.343147,-1.996417,...,-0.275427,-0.756904,-0.150675,-0.811828,-1.152741,0.811543,0.744583,-0.155386,-0.802500,0.708875


In [25]:
def getPiece(row): 
    if row.split_a > grid["split_a"]:
        if row.split_b > grid["split_b"]:
            if row.split_c > grid["split_c"]:
                return 0
            else:
                return 1
        else:
            if row.split_c > grid["split_c"]:
                return 2
            else:
                return 3
    else:
        if row.split_b > grid["split_b"]:
            if row.split_c > grid["split_c"]:
                return 4
            else:
                return 5
        else:
            if row.split_c > grid["split_c"]:
                return 6
            else:
                return 7
        
def generatePrediction(df): # predicts using the classifier from the region
    y_pred = []
    for idx in df.index:
        obs  = df.loc[idx]
        temp = getPiece(obs)
        jdx = clfs[temp].predict(obs.values.reshape(1,-1))
        y_pred.append(jdx[0])
    return pd.Series(y_pred)

## Results

In [26]:
y_pred = generatePrediction(X_test)
print("MAE =", mean_absolute_error(y_test, y_pred))
print("Acc. =", accuracy_score(y_test, y_pred))

MAE = 0.378125
Acc. = 0.653125


In [27]:
confusion_matrix(y_test, y_pred)

array([[  0,   0,   1,   0,   0,   0],
       [  1,   0,  11,   1,   0,   0],
       [  0,   2, 101,  29,   1,   0],
       [  0,   1,  30,  88,   8,   4],
       [  0,   0,   1,  18,  20,   1],
       [  0,   0,   0,   1,   1,   0]])

In [28]:
print(y_pred.value_counts())

5    144
6    137
7     30
8      5
4      3
3      1
dtype: int64


In [29]:
print(y_test.value_counts())

5    133
6    131
7     40
4     13
8      2
3      1
Name: quality, dtype: int64


## Using Ensemble of all the grid pieces

In [30]:
X_train, y_train = train.ix[:,:-1], train['quality']

In [31]:
X_train.head()

,split_a,split_c,split_b,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,-1.611698,-2.497757,-0.469312,-0.530820,-1.612111,-1.883671,-1.550030,-1.108409,-2.409026,1.977169,...,0.737246,0.060633,0.715929,-0.782720,-0.695453,0.867302,-0.444191,-0.422790,-0.298395,0.021376
1,-0.482361,-0.696739,1.403763,-2.586344,-3.579486,-4.026166,-3.829159,-2.871945,-3.524733,1.909679,...,-0.933465,-1.035477,-0.932602,0.747063,-0.060618,-1.311487,1.234117,0.729584,-1.210805,0.243857
2,-1.392553,-2.003326,0.446310,-0.927503,-1.313883,-1.888500,-1.640051,-1.167925,-2.011349,1.154108,...,0.138882,-0.361869,-0.155358,0.368744,0.326022,-0.199981,0.570912,0.143388,-0.591729,0.113832
4,-1.611698,-2.497757,-0.469312,-0.530820,-1.612111,-1.883671,-1.550030,-1.108409,-2.409026,1.977169,...,0.737246,0.060633,0.715929,-0.782720,-0.695453,0.867302,-0.444191,-0.422790,-0.298395,0.021376
5,-1.306524,-2.027766,0.085938,-0.667094,-1.619920,-2.119068,-1.584720,-1.279085,-2.349667,1.925010,...,1.080161,0.403221,0.967208,-0.929893,-0.760095,0.976192,-0.696938,-0.630834,-0.085295,-0.087132


In [32]:
p = X_train.shape[1]

In [33]:
X_test.head()

,split_a,split_c,split_b,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
3,-1.555519,-1.950545,0.914636,2.567142,2.638137,0.799670,-0.859453,-0.566490,-0.615480,0.370947,...,-0.910964,-1.173742,-1.468858,-0.562162,0.333830,-0.369856,0.408950,1.255754,-0.267939,1.780407
6,-1.240276,-1.852070,0.538802,-0.764130,-1.350139,-1.965743,-1.356869,-0.961592,-1.743965,1.267846,...,0.275995,-0.409521,0.138785,-0.185034,-0.029905,0.529878,-0.185296,-0.661687,0.204157,-0.147077
8,-1.934417,-2.710839,-0.515871,-0.471903,-0.875078,-0.917104,-0.529528,-0.085305,-1.090752,0.800381,...,0.329171,0.144408,0.717904,-0.622600,-0.329310,0.585613,0.108161,0.010893,0.014209,-0.307929
11,-1.682804,-2.199949,-0.107956,-0.542672,-0.076116,0.832077,0.785671,0.376709,-0.566937,1.271838,...,0.543039,0.417775,0.546078,0.332745,0.105123,0.141675,0.882439,0.081475,-0.173375,0.213253
12,1.600006,0.367939,-0.970042,-1.433279,-0.481430,-2.585388,-2.699047,-2.634464,0.343147,-1.996417,...,-0.275427,-0.756904,-0.150675,-0.811828,-1.152741,0.811543,0.744583,-0.155386,-0.802500,0.708875


In [34]:
# Expand add predictions from the segmented classifiers to X_train and X_test
for idx, clf in enumerate(clfs):
    temp = clf.predict(X_train.ix[:,:p])
    X_train.insert(X_train.shape[1], 'clf_' + str(idx), temp)
    temp = clf.predict(X_test.ix[:,:p])
    X_test.insert(X_test.shape[1], 'clf_' + str(idx), temp)

In [35]:
clf = LogisticRegression(penalty='l2', C=10, n_jobs=-1)
clf.fit(X_train, y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [36]:
y_pred = clf.predict(X_test)
print("MAE =", mean_absolute_error(y_test, y_pred))
print("Acc. =", accuracy_score(y_test, y_pred))

MAE = 0.3125
Acc. = 0.7


In [37]:
confusion_matrix(y_test, y_pred)

array([[  0,   0,   1,   0,   0,   0],
       [  0,   1,  11,   1,   0,   0],
       [  0,   0, 109,  24,   0,   0],
       [  0,   0,  30,  97,   4,   0],
       [  0,   0,   1,  21,  17,   1],
       [  0,   0,   0,   1,   1,   0]])

In [38]:
print(pd.Series(y_pred).value_counts())

5    152
6    144
7     22
8      1
4      1
dtype: int64


In [39]:
print(y_test.value_counts())

5    133
6    131
7     40
4     13
8      2
3      1
Name: quality, dtype: int64
